In [1]:
!pip install --upgrade plotly

     |████████████████████████████████| 26.5 MB 4.9 MB/s 
  Attempting uninstall: plotly
    Found existing installation: plotly 4.4.1
    Uninstalling plotly-4.4.1:
      Successfully uninstalled plotly-4.4.1


In [37]:
import os

import plotly
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd

In [3]:
plotly.__version__

'5.5.0'

In [52]:
INPUT_PATH = '../../data/3_manually_processed/forms'
OUTPUT_PATH = '../../data/4_content'

## Reading

In [53]:
enem = pd.read_csv(os.path.join(INPUT_PATH, 'enem.csv')).convert_dtypes()
enem.head()

,id,unidade,curso,ano,modalidade,chamada,nota_linguagens,nota_humanas,nota_natureza,nota_matematica,nota_redacao,nota_final,nota_redacao_c1,nota_redacao_c2,nota_redacao_c3,nota_redacao_c4,nota_redacao_c5,redacao
0,4,EACH,Biotecnologia,2020,"Pretos, Pardos e Indígenas (PPI)",<NA>,621.0,625.0,607.0,724.0,720,661.142857,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,4,EACH,Biotecnologia,2020,Escola Pública (EP),<NA>,601.4,629.1,591.7,746.7,960,695.328571,160,200,200,200,200,<NA>
2,2,EACH,Ciências da Natureza (Lic),2019,Escola Pública (EP),<NA>,640.8,697.4,623.1,679.5,820,700.557143,160,180,160,160,160,<NA>
3,12,EACH,Gestão de Políticas Públicas,2017,"Pretos, Pardos e Indígenas (PPI)",<NA>,639.0,698.0,600.0,678.0,800,703.000000,160,160,140,180,160,<NA>
4,12,EACH,Gestão de Políticas Públicas,2019,"Pretos, Pardos e Indígenas (PPI)",<NA>,646.0,680.1,597.0,755.0,760,703.920000,160,200,200,120,80,<NA>


In [54]:
fuvest = pd.read_csv(os.path.join(INPUT_PATH, 'fuvest.csv')).convert_dtypes()
fuvest.head()

,id,unidade,curso,ano,modalidade,chamada,nota_1,nota_2_1,nota_2_2,nota_final,nota_redacao,classificacao_carreira,classificacao_curso,redacao
0,4,EACH,Biotecnologia,2020,Escola Pública (EP),<NA>,51,51.88,31.25,465.988889,30.0,32,<NA>,<NA>
1,4,EACH,Biotecnologia,2020,Escola Pública (EP),<NA>,52,68.00,34.00,532.592593,43.0,7,<NA>,<NA>
2,4,EACH,Biotecnologia,2020,Escola Pública (EP),<NA>,60,55.00,41.67,544.455556,31.5,5,<NA>,<NA>
3,4,EACH,Biotecnologia,2020,Ampla Concorrência (AC),<NA>,57,59.50,49.17,573.344444,32.0,27,<NA>,<NA>
4,4,EACH,Biotecnologia,2020,Ampla Concorrência (AC),<NA>,64,68.00,41.67,602.603704,36.0,20,<NA>,<NA>


## Pre-processing

In [55]:
fuvest.ano = fuvest.ano.fillna(2021)
enem.ano = enem.ano.fillna(2021)

## Analysing

### Support functions

In [56]:
def table_from_df(df, cols=None):
  if cols is None:
    cols = df.columns
    
  data_list = [df[col] for col in cols]
  fig = go.Figure(data=go.Table(
      header=dict(values=cols),
      cells=dict(values=data_list)
  ))
  return fig

In [57]:
def plotly_fig_to_html(fig, path):
  fig.update_layout(margin=dict(
      b=0,
      l=0,
      r=0,
      t=0,
  ))
  fig.write_html(path, include_plotlyjs='cdn', full_html=False)

### Fuvest

In [86]:
for index, row in fuvest[['id', 'ano']].drop_duplicates().iterrows():
  os.makedirs(os.path.join(OUTPUT_PATH, str(row.id), 'fuvest', str(row.ano)))

In [59]:
fuvest_grouped = fuvest.groupby(['id', 'ano', 'modalidade'])
fuvest_grouped.groups

{(0, 2019, 'Escola Pública (EP)'): [61], (0, 2019, 'Pretos, Pardos e Indígenas (PPI)'): [70], (0, 2020, 'Ampla Concorrência (AC)'): [63, 64, 65, 66], (0, 2020, 'Escola Pública (EP)'): [58, 59, 60, 62], (0, 2021, 'Ampla Concorrência (AC)'): [67, 68, 71, 73, 75, 77, 78, 81, 82, 83, 84, 85, 87, 89, 91, 92], (0, 2021, 'Escola Pública (EP)'): [69, 72, 74, 76, 79, 80, 86, 88, 90], (2, 2021, 'Escola Pública (EP)'): [43], (4, 2020, 'Ampla Concorrência (AC)'): [3, 4, 5, 6, 7, 8], (4, 2020, 'Escola Pública (EP)'): [0, 1, 2], (4, 2021, 'Ampla Concorrência (AC)'): [9, 10, 11, 12, 13, 14, 15, 17], (4, 2021, 'Escola Pública (EP)'): [16], (6, 2021, 'Ampla Concorrência (AC)'): [56, 57], (6, 2021, 'Escola Pública (EP)'): [55], (7, 2019, 'Escola Pública (EP)'): [93], (7, 2021, 'Ampla Concorrência (AC)'): [94, 96, 100, 101, 103], (7, 2021, 'Escola Pública (EP)'): [95, 97, 98, 99], (7, 2021, 'Pretos, Pardos e Indígenas (PPI)'): [102], (10, 2019, 'Ampla Concorrência (AC)'): [18], (10, 2020, 'Escola Pública

In [60]:
fuvest_metrics_df = fuvest_grouped.agg(
  min=('nota_final', 'min'),
  max=('nota_final', 'max'), 
  mean=('nota_final', 'mean'),
)
fuvest_metrics_df

min         max        mean
id  ano  modalidade                                                          
0   2019 Escola Pública (EP)               506.481482  506.481482  506.481482
         Pretos, Pardos e Indígenas (PPI)  474.630000  474.630000  474.630000
    2020 Ampla Concorrência (AC)           575.729630  644.618518  609.765741
         Escola Pública (EP)               424.085185  546.666667  480.552778
    2021 Ampla Concorrência (AC)           577.960000  729.810000  649.513750
...                                               ...         ...         ...
174 2020 Escola Pública (EP)               610.370000  642.780000  626.075833
         Pretos, Pardos e Indígenas (PPI)  511.480000  672.590000  575.246667
    2021 Ampla Concorrência (AC)           764.260000  764.260000  764.260000
         Escola Pública (EP)               643.700000  643.700000  643.700000
177 2021 Escola Pública (EP)               688.520000  688.520000  688.520000

[487 rows x 3 columns]

#### Metrics plot

In [61]:
example_metrics = fuvest_metrics_df.loc[0].reset_index()
example_metrics

,ano,modalidade,min,max,mean
0,2019,Escola Pública (EP),506.481482,506.481482,506.481482
1,2019,"Pretos, Pardos e Indígenas (PPI)",474.630000,474.630000,474.630000
2,2020,Ampla Concorrência (AC),575.729630,644.618518,609.765741
3,2020,Escola Pública (EP),424.085185,546.666667,480.552778
4,2021,Ampla Concorrência (AC),577.960000,729.810000,649.513750
5,2021,Escola Pública (EP),479.440000,579.260000,527.895556


In [62]:
molten_example_metrics = pd.melt(example_metrics,
                                 id_vars=['ano', 'modalidade'],
                                 value_vars=['min', 'max', 'mean'],
                                 var_name='metric',
                                 value_name='nota')
molten_example_metrics

,ano,modalidade,metric,nota
0,2019,Escola Pública (EP),min,506.481482
1,2019,"Pretos, Pardos e Indígenas (PPI)",min,474.630000
2,2020,Ampla Concorrência (AC),min,575.729630
3,2020,Escola Pública (EP),min,424.085185
4,2021,Ampla Concorrência (AC),min,577.960000
5,2021,Escola Pública (EP),min,479.440000
6,2019,Escola Pública (EP),max,506.481482
7,2019,"Pretos, Pardos e Indígenas (PPI)",max,474.630000
8,2020,Ampla Concorrência (AC),max,644.618518
9,2020,Escola Pública (EP),max,546.666667


In [63]:
px.line(
  molten_example_metrics,
  x='ano', y='nota',
  color='metric',
  facet_row='modalidade',
  markers=True,
)

In [64]:
figs = []
for id, df in fuvest_metrics_df.groupby('id'):
  molten_df = pd.melt(df.reset_index(),
                      id_vars=['ano', 'modalidade'],
                      value_vars=['min', 'max', 'mean'],
                      var_name='metric',
                      value_name='nota')
  fig = px.line(
    molten_df,
    x='ano', y='nota',
    color='metric',
    facet_row='modalidade',
    markers=True, # for line chart
    # barmode='group', # for bar chart
  )
  figs.append(fig)

print(len(figs))
figs[0].show()

124


In [65]:
figs[102].show()

In [66]:
def plot_fuvest_metric_evolution(fuvest_metrics_df):
  for id, df in fuvest_metrics_df.groupby('id'):
    molten_df = pd.melt(df.reset_index(),
                        id_vars=['ano', 'modalidade'],
                        value_vars=['min', 'max', 'mean'],
                        var_name='metric',
                        value_name='nota')
    fig = px.line(
      molten_df,
      x='ano', y='nota',
      color='metric',
      facet_row='modalidade',
      markers=True, # for line chart
      # barmode='group', # for bar chart
    )

    plotly_fig_to_html(
      fig,
      os.path.join(OUTPUT_PATH, str(id), 'fuvest',
                   'metric_evolution.html'))

In [87]:
plot_fuvest_metric_evolution(fuvest_metrics_df)

#### Metrics table for the latest year

In [78]:
for id, df in fuvest_metrics_df.reset_index().groupby('id'):
  print(id)
  print(df[['ano', 'modalidade', 'min', 'mean', 'max']])

0
    ano                        modalidade         min        mean         max
0  2019               Escola Pública (EP)  506.481482  506.481482  506.481482
1  2019  Pretos, Pardos e Indígenas (PPI)  474.630000  474.630000  474.630000
2  2020           Ampla Concorrência (AC)  575.729630  609.765741  644.618518
3  2020               Escola Pública (EP)  424.085185  480.552778  546.666667
4  2021           Ampla Concorrência (AC)  577.960000  649.513750  729.810000
5  2021               Escola Pública (EP)  479.440000  527.895556  579.260000
2
    ano           modalidade     min    mean     max
6  2021  Escola Pública (EP)  352.78  352.78  352.78
4
     ano               modalidade         min        mean         max
7   2020  Ampla Concorrência (AC)  573.344444  645.005556  735.555556
8   2020      Escola Pública (EP)  465.988889  514.345679  544.455556
9   2021  Ampla Concorrência (AC)  616.480000  674.373750  823.330000
10  2021      Escola Pública (EP)  618.330000  618.330000  618

In [83]:
metric_tables = []
# TODO add latest avaliable year to the title of the table
for id, df in fuvest_metrics_df.reset_index().groupby('id'):
  latest_metrics_df = df.loc[df.ano == df.ano.max()]
  metric_tables.append(table_from_df(latest_metrics_df,
                                     ['modalidade', 'min', 'mean', 'max']))

print(len(metric_tables))
metric_tables[0].show()

124


In [88]:
def plot_fuvest_latest_metrics_table(fuvest_metrics_df):
  # TODO add latest avaliable year to the title of the table
  for id, df in fuvest_metrics_df.reset_index().groupby('id'):
    latest_metrics_df = df.loc[df.ano == df.ano.max()]
    plotly_fig_to_html(
      table_from_df(latest_metrics_df,
                    ['modalidade', 'min', 'mean', 'max']),
      os.path.join(OUTPUT_PATH, str(id), 'fuvest',
                   'latest_metrics.html'))

In [89]:
plot_fuvest_latest_metrics_table(fuvest_metrics_df)

#### Data tables

In [71]:
TABLE_COL_NAMES = [
  'chamada',
  'nota_1',
  'nota_2_1',
  'nota_2_2',
  'nota_final',
  'nota_redacao',
  'classificacao_carreira',
  'classificacao_curso',
]
data_tables = []

for id, df in fuvest.groupby('id'):
  df = df.reset_index()
  for ano in df['ano'].unique():
    for modalidade in df['modalidade'].unique():
      sliced_df = df[(df['ano'] == ano)
                     & (df['modalidade'] == modalidade)]
      if not sliced_df.empty:
        sliced_df = sliced_df.sort_values('nota_final', ascending=False)
        full_df = sliced_df.astype(str)
        data_tables.append(table_from_df(full_df, TABLE_COL_NAMES))
print(len(data_tables))
data_tables[0].show()

487


In [90]:
def plot_fuvest_data_tables(fuvest_df, cols):
  for id, df in fuvest.groupby('id'):
    df = df.reset_index()
    for ano in df['ano'].unique():
      for modalidade in df['modalidade'].unique():
        sliced_df = df[(df['ano'] == ano)
                      & (df['modalidade'] == modalidade)]
        if not sliced_df.empty:
          sliced_df = sliced_df.sort_values('nota_final', ascending=False)
          full_df = sliced_df.astype(str)
          plotly_fig_to_html(
            table_from_df(full_df, cols),
            os.path.join(OUTPUT_PATH, str(id), 'fuvest',
                         str(ano), f'{modalidade}.html')  
          )

In [91]:
plot_fuvest_data_tables(fuvest, TABLE_COL_NAMES)